# Test of agent_framework.py

In [1]:
import pyterrier as pt
import pyterrier_rag.readers
from pyterrier_rag.backend import HuggingFaceBackend

In [2]:
import agent_framework
from agent_framework import R1Searcher
from agent_framework import SearchR1

INFO 08-15 11:21:27 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 08-15 11:21:27 [__init__.py:239] Automatically detected platform cuda.


In [3]:
sparse_index = pt.Artifact.from_hf('pyterrier/ragwiki-terrier')
bm25_ret = pt.rewrite.tokenise() >> sparse_index.bm25(include_fields=['docno', 'text', 'title'], threads=5, verbose = True) >> pt.rewrite.reset() >> pt.debug.print_rows()

Java started (triggered by tokenise) and loaded: pyterrier.java, pyterrier.terrier.java [version=5.11 (build: craig.macdonald 2025-01-13 21:29), helper_version=0.0.8]
/opt/miniconda3/lib/python3.10/site-packages/pyterrier/terrier/retriever.py:219: UserWarning: Multi-threaded retrieval is experimental, YMMV.
  warn(
/opt/miniconda3/lib/python3.10/site-packages/pyterrier/terrier/retriever.py:226: UserWarning: Upgrading indexref /mnt/resources/pyterrier-cache/artifacts/ebfd80cc597a31719f11ab5cd11ad8f441bc460f760c82ff66413ba9fb06943f/data.properties to be concurrent
  warn(


## r1_searcher

In [4]:
r1_searcher = R1Searcher(
    retriever = bm25_ret
)

INFO 08-15 11:21:50 [config.py:717] This model supports multiple tasks: {'classify', 'generate', 'score', 'embed', 'reward'}. Defaulting to 'generate'.
WARNING 08-15 11:21:50 [cuda.py:93] To see benefits of async output processing, enable CUDA graph. Since, enforce-eager is enabled, async output processor cannot be used
INFO 08-15 11:21:50 [llm_engine.py:240] Initializing a V0 LLM engine (v0.8.5.post1) with config: model='XXsongLALA/Qwen-2.5-7B-base-RAG-RL', speculative_config=None, tokenizer='XXsongLALA/Qwen-2.5-7B-base-RAG-RL', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.bfloat16, max_seq_len=2048, download_dir=None, load_format=auto, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=True, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='auto', reasoning_backend=N

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

[R1Searcher] transformers backend ready: XXsongLALA/Qwen-2.5-7B-base-RAG-RL on cuda:0


In [6]:
res = r1_searcher.search("Who is the mother of the president of China?")
res

TerrierRetr(BM25): 100%|██████████| 1/1 [00:00<00:00,  2.69q/s]


,qid,docid,docno,text,title,rank,score,query
0,1-1,10466655,10466655,Spouse of the President of China The Spouse of...,"""Spouse of the President of China""",0,25.283915,current president of China
1,1-1,10466656,10466656,Spouse of the President of China The Spouse of...,"""Spouse of the President of China""",1,25.283915,current president of China


TerrierRetr(BM25): 100%|██████████| 1/1 [00:00<00:00,  5.50q/s]


,qid,docid,docno,text,title,rank,score,query
0,1-2,16233533,16233533,A Mother's Ordeal A Mother's Ordeal: One Woman...,"""A Mother's Ordeal""",0,24.582340,president of China mother
1,1-2,19803511,19803511,"father Richard Lee grew up in Shanghai, but wa...","""Handel Lee""",1,21.893824,president of China mother


,qid,query,context,search_history,search_iterations,qanswer,output,stop_reason
0,1,Who is the mother of the president of China?,"The User asks a question, and the Assistant so...","[current president of China, president of Chin...",2,Grace Betty Soong,"To solve this question, I need to find out the...",Got answer


In [7]:
res.output[0]

"To solve this question, I need to find out the name of the mother of the president of China. \n\nStep 1: Identify the current president of China.\nStep 2: Determine the name of the president's mother.\n\n**Step 1: Identify the current president of China.**\n\nI'll use a search query to find out:\n\n<|begin_of_query|> current president of China <|end_of_query|>\n\n </think>\n\n<|begin_of_query|> president of China mother <|end_of_query|>\n\n </think>\n\n<answer> Grace Betty Soong </answer>"

In [13]:
res = r1_searcher.search("Who is the No.1 driver of Ferrari Fomula 1 team?")
res

TerrierRetr(BM25): 100%|██████████| 1/1 [00:00<00:00,  3.16q/s]


,qid,docid,docno,text,title,rank,score,query
0,1-1,8085531,8085531,replaced until when Ferrari began using the D5...,"""Ferrari Tipo 500""",0,27.007631,Ferrari Formula 1 team No.1 driver
1,1-1,8085528,8085528,Ferrari Tipo 500 The Ferrari 500 was a Formula...,"""Ferrari Tipo 500""",1,25.875845,Ferrari Formula 1 team No.1 driver


TerrierRetr(BM25): 100%|██████████| 1/1 [00:00<00:00,  3.85q/s]


,qid,docid,docno,text,title,rank,score,query
0,1-2,16936416,16936416,Scuderia went on to win the Constructors Champ...,"""Enzo Ferrari""",0,21.556887,Ferrari Fomula 1 team No.1 driver
1,1-2,7446348,7446348,Championship. Before the final race at the str...,"""1991 Formula One World Championship""",1,21.323496,Ferrari Fomula 1 team No.1 driver


,qid,query,context,search_history,search_iterations,qanswer,output,stop_reason
0,1,Who is the No.1 driver of Ferrari Fomula 1 team?,"The User asks a question, and the Assistant so...","[Ferrari Formula 1 team No.1 driver, Ferrari F...",2,Fernando Alonso and Kimi Räikkönen,"To solve this question, I need to find out who...",Got answer


In [14]:
res.output[0]

'To solve this question, I need to find out who the No.1 driver of the Ferrari Formula 1 team is. \n\nStep 1: Identify the current No.1 driver of the Ferrari Formula 1 team.\n\nStep 2: Use external knowledge to search for information about the No.1 driver of the Ferrari Formula 1 team.\n\n <|begin_of_query|> Ferrari Formula 1 team No.1 driver <|end_of_query|>\n\n \n\nFrom the provided information, I can see that Ferrari Fomula 1 team has had several drivers over the years, including Fernando Alonso, Kimi Räikkönen, Luca Badoer, Giancarlo Fisichella, Marc Gené, Sergio Pérez, and others. However, the question specifically asks for the "No.1 driver" of the Ferrari Fomula 1 team.\n\nTo find out the No.1 driver, I will perform a search using the following query:\n\n<|begin_of_query|> Ferrari Fomula 1 team No.1 driver <|end_of_query|>\n\n </think> \n\nBased on the provided information, it is not explicitly stated who the No.1 driver of the Ferrari Formula 1 team is. However, from the context

In [10]:
r1_searcher.generate([res.iloc[0]['context']])

[' </think>\n\n<answer> Alberto Ascari </answer>']